In [ ]:
# Importation des bibliothèques nécessaires
from datetime import datetime
import re

# Fonction pour parser la date et l'heure au format de connexion.log
def parse_datetime(date_str):
    return datetime.strptime(date_str, "%d/%m/%y %H:%M")

# Tâche 1 : Extraire les utilisateurs uniques et les sauvegarder dans utilisateurs.txt
def extract_users(log_file, output_file):
    users = set()
    with open(log_file, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) == 3:
                users.add(parts[1])
    
    with open(output_file, 'w') as f:
        for user in sorted(users):
            f.write(user + '\n')

# Tâche 2 : Trouver les utilisateurs qui se connectent en dehors des heures de bureau (8h-18h)
def find_suspicious_connections(log_file):
    suspicious = []
    with open(log_file, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) == 3:
                ip, user, date_time = parts
                dt = parse_datetime(date_time)
                hour = dt.hour
                # Vérifier si la connexion est en dehors de 8h-18h
                if hour < 8 or hour >= 18:
                    suspicious.append((user, ip, date_time))
    return suspicious

# Tâche 3 : Trouver les utilisateurs se connectant à des IPs malveillantes et compter les occurrences
def find_malicious_connections(log_file, warning_file, output_file):
    # Lire les IPs malveillantes
    malicious_ips = set()
    try:
        with open(warning_file, 'r') as f:
            for line in f:
                malicious_ips.add(line.strip())
    except FileNotFoundError:
        print(f"Le fichier {warning_file} n'existe pas. Création d'un suspect.txt vide.")
        malicious_ips = set()
    
    # Compter les connexions aux IPs malveillantes par utilisateur
    user_counts = {}
    with open(log_file, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) == 3:
                ip, user, _ = parts
                if ip in malicious_ips:
                    user_counts[user] = user_counts.get(user, 0) + 1
    
    # Écrire les résultats dans suspect.txt
    with open(output_file, 'w') as f:
        for user, count in sorted(user_counts.items()):
            f.write(f"{user},{count}\n")

# Exécution principale
def main():
    log_file = 'connexion.log'
    warning_file = 'warning.txt'
    
    # Tâche 1 : Générer utilisateurs.txt
    extract_users(log_file, 'utilisateurs.txt')
    print("Tâche 1 : utilisateurs.txt généré")
    
    # Tâche 2 : Afficher les connexions suspectes (en dehors des heures de bureau)
    suspicious_connections = find_suspicious_connections(log_file)
    print("\nTâche 2 : Connexions en dehors des heures de bureau (8h-18h) :")
    for user, ip, date_time in suspicious_connections:
        print(f"Utilisateur : {user}, IP : {ip}, Date/Heure : {date_time}")
    
    # Tâche 3 : Générer suspect.txt
    find_malicious_connections(log_file, warning_file, 'suspect.txt')
    print("\nTâche 3 : suspect.txt généré")

if __name__ == "__main__":
    main()


            

